In [ ]:
import pandas as pd, numpy as np, lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from joblib import Parallel, delayed
import os, itertools, warnings, json

DATA_FILE = "features_v1_tail.csv"
TARGET    = "return_72h"
QUANTS    = [0.05, 0.25, 0.50, 0.75, 0.95]

df = (pd.read_csv(DATA_FILE, parse_dates=["timestamp"])
        .sort_values(["token", "timestamp"])
        .reset_index(drop=True))

cat_cols = ["day_of_week","momentum_bucket", "extreme_flag1", "tail_asym","vol_regime", "token"]
num_cols = [c for c in df.columns
            if c not in cat_cols + ["timestamp", TARGET]]

# one-hot → dense matrix; LightGBM handles NaN in numeric naturally
pre = ColumnTransformer([
        ("cats", OneHotEncoder(drop="first",
                               handle_unknown="ignore",
                               sparse_output=False), cat_cols)
      ],
      remainder="passthrough")

In [ ]:
TRAIN, CAL, TEST = 120, 24, 6     # bars (~60d, 12d, 3d)

def rolling_splits(idx):
    for start in range(0, len(idx) - (TRAIN + CAL + TEST) + 1, TEST):
        tr = idx[start : start + TRAIN]
        cal = idx[start + TRAIN : start + TRAIN + CAL]
        te = idx[start + TRAIN + CAL : start + TRAIN + CAL + TEST]
        if len(te) == TEST:
            yield tr, cal, te

In [ ]:
def fit_one_fold(g, tr_idx, cal_idx, te_idx):
    """
    Fit LightGBM-quantile on one rolling window and return
    • fold_pred : list[dict]  (row-level predictions)
    • fold_res  : list[dict]  (fold-level pinball loss)
    """
    # ── matrices ─────────────────────────────────────────
    X_tr  = pre.fit_transform(g.loc[tr_idx, cat_cols + num_cols])
    y_tr  = g.loc[tr_idx, TARGET].values
    X_cal = pre.transform(g.loc[cal_idx, cat_cols + num_cols])
    y_cal = g.loc[cal_idx, TARGET].values
    X_te  = pre.transform(g.loc[te_idx, cat_cols + num_cols])
    y_te  = g.loc[te_idx, TARGET].values

    token_id = g["token"].iloc[0]      # ← safe token label

    fold_pred, fold_res = [], []

    for tau in QUANTS:
        mdl = lgb.LGBMRegressor(
            objective="quantile", alpha=tau,
            n_estimators=500, learning_rate=0.05,
            max_depth=-1, subsample=0.9, colsample_bytree=0.9,
            min_child_samples=20, random_state=42
        )
        mdl.fit(X_tr, y_tr)

        # ── base predictions ─────────────────────────────
        cal_hat = mdl.predict(X_cal)
        te_hat  = mdl.predict(X_te)

        # ── split-conformal adjustment ───────────────────
        resid = y_cal - cal_hat
        if tau < 0.5:
            adj = np.quantile(np.maximum(resid, 0), 1 - tau)
            te_adj = te_hat - adj
        elif tau > 0.5:
            adj = np.quantile(np.maximum(-resid, 0), 1 - (1 - tau))
            te_adj = te_hat + adj
        else:                       # τ = 0.50
            te_adj = te_hat

        # ── per-row predictions ──────────────────────────
        fold_pred.extend({
            "timestamp": g.loc[i, "timestamp"],
            "token":     token_id,
            "tau":       tau,
            "y_true":    yt,
            "y_pred":    yp
        } for i, yt, yp in zip(te_idx, y_te, te_adj))

        # ── fold-level pinball loss ──────────────────────
        err = y_te - te_adj
        pin = np.maximum(tau*err, (tau-1)*err).mean()
        fold_res.append({
            "token":   token_id,
            "tau":     tau,
            "pinball": pin
        })

    return fold_pred, fold_res


In [ ]:
def run_token(tok, grp):
    preds, metrics = [], []
    for tr, cal, te in rolling_splits(grp.index):
        p, m = fit_one_fold(grp, tr, cal, te)
        preds.extend(p); metrics.extend(m)
    return preds, metrics

n_jobs = max(os.cpu_count()-1, 1)
results = Parallel(n_jobs=n_jobs, verbose=5)(
    delayed(run_token)(tok, grp.reset_index(drop=True))
    for tok, grp in df.groupby("token"))

preds   = list(itertools.chain.from_iterable(r[0] for r in results))
metrics = list(itertools.chain.from_iterable(r[1] for r in results))

pd.DataFrame(preds).to_csv("stage7_lgb_preds.csv", index=False)
pd.DataFrame(metrics).to_csv("stage7_lgb_pinball.csv", index=False)

print(pd.DataFrame(metrics)
        .groupby("tau")["pinball"].mean()
        .round(4))


[Parallel(n_jobs=23)]: Using backend LokyBackend with 23 concurrent workers.


KeyboardInterrupt: 